In [1]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw

In [ ]:
def smoke_mask(self, dataset_dir): 
       
        # Add classes. We have only one class to add.
    self.add_class("smoke", 1, "smoke") 

    dataset_dir = os.path.join(dataset_dir) 
        
        # Load annotations
        # VGG Image Annotator saves each image in the form:
        # { 'filename': '28503151_5b5b7ec140_b.jpg',
        #   'regions': {
        #       '0': {
        #           'region_attributes': {},
        #           'shape_attributes': {
        #               'all_points_x': [...],
        #               'all_points_y': [...],
        #               'name': 'polygon'}},
        #       ... more regions ...
        #   },
        #   'size': 100202
        # }

    annotations = json.load(open(os.path.join(dataset_dir, "via_region_data.json")))
    annotations = list(annotations.values())  # don't need the dict keys
        
    print(annotations) 

        # The VIA tool saves images in the JSON even if they don't have any
        # annotations. Skip unannotated images.
    annotations = [a for a in annotations if a['regions']]

        # Add images
    for a in annotations:
            # Get the x, y coordinaets of points of the polygons that make up
            # the outline of each object instance. There are stores in the
            # shape_attributes 
        polygons = [r['shape_attributes'] for r in a['regions'].values()]

            # load_mask() needs the image size to convert polygons to masks.
            
        image_path = os.path.join(dataset_dir, a['filename'])
        image = skimage.io.imread(image_path)
        height, width = image.shape[:2]

        image_id=a['filename']  # use file name as a unique image id
        path=image_path,
        width=width
        height=height,
        polygons=polygons 

def load_mask(self, image_id):
        """Generate instance masks for an image.
       Returns:
        masks: A bool array of shape [height, width, instance count] with
            one mask per instance.
        """

    image_info = self.image_info[image_id] 

        # Convert polygons to a bitmap mask of shape
        # [height, width, instance_count]
    info = self.image_info[image_id]
    mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
    for i, p in enumerate(info["polygons"]):
            # Get indexes of pixels inside the polygon and set them to 1
        rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
        mask[rr, cc, i] = 1

        # Return mask, and array of class IDs of each instance. Since we have
        # one class ID only, we return an array of 1s
    return mask, np.ones([mask.shape[-1]], dtype=np.int32)